In [1]:
# Get autocomplete to work
%config Completer.use_jedi = False

# Ensure external Python files are refreshed when reimporting things
%load_ext autoreload
%autoreload 2

In [2]:
from load_functions import load_text
    
text_input = load_text(day=9)
print(len(text_input))
text_input[:5]

100


['9876543298765698956789545678992103976545976323678910997879876432123679765323567894679434987654596521',
 '7998789129854987545697658789789212985439765454567899876568976569012489874212779943598929976653987432',
 '6429898934943496434589769894658924976929898765679969965345698798923456976433567892456998765432196543',
 '5210987899874987645678979943547895989899989898789357893234599987894678986547678901349899896563987656',
 '4329896798765797657889989652126789998798967999899969954345989876789989797698789316498766987854598787']

Convert to list of lists

In [16]:
grid = []
for t in text_input:
    grid.append([int(n) for n in list(t)])
    
print(len(grid[0]))
print(grid[0][:10], grid[1][:10], sep='\n')

100
[9, 8, 7, 6, 5, 4, 3, 2, 9, 8]
[7, 9, 9, 8, 7, 8, 9, 1, 2, 9]


# Problem 

#### Part 1: Figure out how many low points there are, and add 1 to each low point score

A low point is a number that is surrounded by numbers that are higher than it, or edges

In [47]:
risk_score = 0
low_point_locs = [] # Used for part 2

def get_adjacent_neighbours(grid, row, col):
    
    # Initialise values
    neighbours = []
    num = grid[row][col]
    row_threshold = len(grid)
    col_threshold = len(grid[0])
    
    # Get our offsets to use
    row_offsets = [row-1, row+1]
    col_offsets = [col-1, col+1]

    # Then for our row and column offsets, check they are in range, if so - add to list of neighbours
    for row_offset in row_offsets:
        if row_offset >= 0 and row_offset < row_threshold:
            neighbours.append([grid[row_offset][col], row_offset, col])
    
    for col_offset in col_offsets:
        if col_offset >= 0 and col_offset < col_threshold:
            neighbours.append([grid[row][col_offset], row, col_offset])
        
    return neighbours

# Check neighbours are all larger
for row in range(len(grid)):
    for col in range(len(grid[0])):
        num = grid[row][col]
        neighbours = get_adjacent_neighbours(grid, row, col)
        
        # If neighbours are all larger, append it to our list to use in part 2, and add to our risk score
        if all([num < n[0] for n in neighbours]):
            low_point_locs.append([num, row, col])
            risk_score += (num + 1)
        
print('Day 9, part 1:', risk_score)

Day 9, part 1: 514


#### Part 2: Once you've found a low point, scan to find the basin size

In [93]:
basin_sizes = []
import numpy as np

def get_co_ord_identifier(row, col):
    
    return str(row) + '-' + str(col)


# Declare recursive function
def basin_size_scan(grid, row, col, basin_co_ords):
    
    # Then scan for adjacent neighbours
    neighbours = get_adjacent_neighbours(grid, row, col)
    
    # Check neighbour isn't in existing basin co-ord list
    new_neighbours = [n for n in neighbours if get_co_ord_identifier(row=n[1], col=n[2]) not in basin_co_ords]
    
    # Filter out neighbours with value < current value, and value = 9
    new_neighbours = [n for n in new_neighbours if n[0] > grid[row][col] and n[0] < 9]
    
    if len(new_neighbours) > 0:
        for n in new_neighbours:
            
            # Add to scanned co-ords
            basin_co_ords.append(get_co_ord_identifier(n[1], n[2]))
            
            # Recursion
            basin_size_scan(grid, n[1], n[2], basin_co_ords)
            
    return basin_co_ords

# Used to track all basin sizes
basin_sizes = []

for low_point in low_point_locs:
    
    # Initialise list with first co-ordinates
    basin_co_ords = [get_co_ord_identifier(low_point[1], low_point[2])]
    
    # Then perform recursive search, making sure to de-dupe any co-ordinates that may be simultaneously searched
    basin_co_ords = np.unique(basin_size_scan(grid, 
                                              low_point[1], 
                                              low_point[2], 
                                              basin_co_ords))

    # Add size to list
    basin_sizes.append(len(basin_co_ords))
    

# Sort list
basin_sizes_sorted = list(reversed(sorted(basin_sizes)))

print('Day 9, part 2:', basin_sizes_sorted[0] * basin_sizes_sorted[1] * basin_sizes_sorted[2])

Day 9, part 2: 1103130
